# Ticket report


In [82]:
import random
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

from datetime import date
import src.compute.changelogs as changelogs
from ipywidgets import interact
from src.db.utils import SnowflakeWrapper
from src.compute.utils import Interval
from src.compute.tickets import get_tickets

conn = SnowflakeWrapper.create_snowflake_connection()
sw = SnowflakeWrapper(conn)

label_map = {
    "metrics": {
        "Average hours": "AVG_HOUR",
        "Average days": "AVG_DAYS",
        "Degree of cycling": "DegreeOfCycling",
    },
    "dims": {
        "Ticket Status": "MERGED_STATUS",
        "Ticket Type": "IssueType",
        "Ticket Priority": "IssuePriority",
        "No color": None
    }
}

In [83]:
# Inter-dependency of the dimensions

def plot_distribution_counts(x_col, color_by, date_from, date_to, max_days):
    current_interval = Interval(date_from, date_to)
    plot_df = get_tickets(sw, current_interval, max_days=max_days)
    return px.histogram(
        plot_df,
        x=label_map["dims"][x_col],
        color=label_map["dims"][color_by],
        barmode="group",
        title=f"There are {len(plot_df.index)} resolved tickets on the interval {current_interval.pretty_str()}",
        hover_data=plot_df.columns,
        labels = {
            label_map["dims"][x_col]: x_col,
            label_map["dims"][color_by]: color_by,
            "count": "Number of tickets"
        }
    )

_ = interact(
        plot_distribution_counts,
        x_col=widgets.Dropdown(options=label_map["dims"].keys(), description="Metric"),
        color_by=widgets.Dropdown(options=label_map["dims"].keys(), description="Colour by"),
        date_from=widgets.DatePicker(value=pd.to_datetime('2019-10-01'), description="Date from"),
        date_to=widgets.DatePicker(value=pd.to_datetime('2020-01-01'), description="Date to"),
        max_days=widgets.IntSlider(value=10, min=0, max=30, description="Max days to resolve")
    )

interactive(children=(Dropdown(description='Metric', options=('Ticket Status', 'Ticket Type', 'Ticket Priority…

In [84]:
# Metric distribution by different dimensions

def plot_metric_counts(x_col, color_by, date_from, date_to, max_days):
    current_interval = Interval(date_from, date_to)
    plot_df = get_tickets(sw, current_interval, max_days=max_days)
    bins = int(plot_df[label_map["metrics"][x_col]].max() - plot_df[label_map["metrics"][x_col]].min()) + 1
    max_bins = 30
    if label_map["metrics"][x_col] == "AVG_HOUR":
        bins = int(bins/8)
    if bins > max_bins:
        plot_df[label_map["metrics"][x_col]] = plot_df[label_map["metrics"][x_col]].apply(lambda x: min(max_bins, x))
        bins = max_bins
    # TODO: add fig.update_layout(bargroupgap=0.01)
    return px.histogram(
        plot_df,
        x=label_map["metrics"][x_col],
        color=label_map["dims"][color_by],
        nbins=bins,
        barmode="group",
        title=f"There are {len(plot_df.index)} resolved tickets on the interval {current_interval.pretty_str()}",
        hover_data=plot_df.columns,
        template="ggplot2",
        labels = {
            label_map["metrics"][x_col]: x_col,
            label_map["dims"][color_by]: color_by,
            "count": "Number of tickets"
        }
    )


_ = interact(
        plot_metric_counts,
        x_col=widgets.Dropdown(options=label_map["metrics"].keys(), description="Metric"),
        color_by=widgets.Dropdown(options=label_map["dims"].keys(), description="Colour by"),
        date_from=widgets.DatePicker(value=pd.to_datetime('2019-10-01'), description="Date from"),
        date_to=widgets.DatePicker(value=pd.to_datetime('2020-01-01'), description="Date to"),
        max_days=widgets.IntSlider(value=10, min=0, max=30, description="Max days to resolve")
    )

interactive(children=(Dropdown(description='Metric', options=('Average hours', 'Average days', 'Degree of cycl…

In [ ]:
dimensions = {
    "Transition": "Transition",
    "Issue type": "IssueType",
    "Issue priority": "IssuePriority"
}

# Ticket transitions
def plot_jira_activity(by, date_from, date_to, max_plots):
    active_interval = Interval(date_from, date_to)
    current_dim = dimensions[by]
    by_status = current_dim == "Transition"
    by_issue_type = current_dim == "IssueType"
    by_issue_priority = current_dim == "IssuePriority"
    plot_df = changelogs.transition_frequency(sw, interval=active_interval, limit=-1, by_status=by_status, by_issue_priority=by_issue_priority, by_issue_type=by_issue_type, by_week=True, order="ASC")
    max_val, min_val  = plot_df["WeekOfYear"].max(), plot_df["WeekOfYear"].min()
    dimension_occurences = sorted([ (transition, plot_df.loc[plot_df[current_dim] == transition, "TotalTransitions"].sum()) for transition in (plot_df[current_dim].unique())], key=lambda x: x[1], reverse=True)
    fig = go.Figure()
    all_df = plot_df.groupby(by=["Year", "WeekOfYear"]).agg(['sum'], axis="columns").drop(columns=[current_dim]).reset_index()
    fig.add_trace(
        go.Scatter(
            x=all_df["WeekOfYear"],
            y=all_df["TotalTransitions"]["sum"],
            mode='lines+markers',
            name=f'All transitions = [{plot_df["TotalTransitions"].sum()}]'
        )
    )
    for dimension_val, cum_sum in dimension_occurences[:max_plots]:
        tr_data = plot_df.loc[plot_df[current_dim] == dimension_val]
        fig.add_trace(
            go.Scatter(
                x=tr_data["WeekOfYear"],
                y=tr_data["TotalTransitions"],
                mode='lines+markers',
                name=f"{dimension_val}, overall: [{cum_sum}]",
            )
        )
    fig.update_layout(
        title="Ticket transition frequency by week",
        xaxis = dict(
            range=[min_val - 1, max_val + 1],  # sets the range of xaxis
            tickmode='linear'
        ),
    )
    return fig

_ = interact(
    plot_jira_activity,
    by=widgets.Dropdown(options=dimensions.keys(), description="Dimension"),
    date_from=widgets.DatePicker(value=pd.to_datetime('2019-10-01'), description="Date from"),
    date_to=widgets.DatePicker(value=pd.to_datetime('2020-01-01'), description="Date to"),
    max_plots=widgets.IntSlider(value=5, min=1, max=10, description="Max graphs to plot"),
)

interactive(children=(Dropdown(description='Dimension', options=('Transition', 'Issue type', 'Issue priority')…